<a href="https://colab.research.google.com/github/adautofbn/ri_labs/blob/master/lab06/modelo_vetorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [88]:
import pandas as pd
import numpy as np
import nltk
import re
import collections
import bisect
from nltk.tokenize import RegexpTokenizer

nltk.download('stopwords')
result = pd.read_csv('https://raw.githubusercontent.com/adautofbn/ri_labs/master/lab06/results.csv')
                                            # Resultados adquiridos do site brasil-247 em abril de 2019

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 1. Reconstruir o índice considerando o conjunto de dados que indicamos. Esses são os dados coletados por Bernardi e os estaremos usando para facilitar a correção da atividade. Se você já estiver usando esses dados não precisa reconstruir o índice;

In [0]:
tknz = RegexpTokenizer(r'([A-Za-zÁáÉéÍíÓóÚúÃãÕõÇçÂâÊê]{3,27})')
stopwords = nltk.corpus.stopwords.words('portuguese') 
indexes = {}
n = 0

for text in result.text:
  words = [word for word in tknz.tokenize(text.lower())
           if not bool(re.search(r'\d', word))
           and word not in stopwords and len(word) >= 3]  
  n += 1
  for t in words:
    if t not in indexes.keys():
      indexes[t] = []
    indexes[t].append(n)
    
for elem in indexes.items():
  d = dict(collections.Counter(elem[1]))
  indexes[elem[0]] = list(d.items())

## 2. Refinar o índice invertido de forma a também incluir o IDF (inverse document frequency) de cada termo do dicionário; 

In [0]:
M = result.text.count()
for word in indexes:
  k = len(indexes[word])
  IDF = round(np.log((M+1)/k),2)
  indexes[word].append(IDF)

## 3. Implementar as seguintes versões do modelo vetorial:

### 3.1. Representação binária

In [0]:
def binary_vsm(query, document):
  score = 0
  query_tokens = query.split()
  doc_tokens = document.split()
  for token in query_tokens:
    score += (token in doc_tokens)
  return score

### 3.2. TF (lembre-se que esse modelo já está implementado)

In [0]:
def tf_vsm(query, document):
  score = 0
  doc_tokens = document.split()
  query_tokens = query.split()
  
  for token in doc_tokens:
    score += (token in query_tokens)
  
  return score

### 3.3 TF-IDF

In [0]:
def tfidf_vsm(query, document):
  score = 0
  doc_tokens = document.split()
  query_tokens = query.split()
  
  for token in doc_tokens:
    if(token in indexes):    
      score += (token in query_tokens) * indexes[token][-1]
  
  return round(score,2)

### 3.4 BM25 (não usaremos Okapi já que os documentos não tem grande variação de tamanho)

In [0]:
def bm25_vsm(query, document, n):
  score = 0
  doc_tokens = document.split()
  query_tokens = query.split()
  
  for query_word in query_tokens:
    k = len(indexes[query_word])
    cwd = (query_word in doc_tokens)
    dfw = 0
    for doc,score in indexes[query_word][:-1]:
      if doc == n:
        dfw = score
    for doc_word in doc_tokens:
      cwq = (doc_word in query_tokens)
      if dfw > 0:
        score += cwq * (((k+1)*cwd) / (cwd + k)) * np.log((M+1)/dfw)
  
  return round(score,2)

## 4. Execute os algoritmos separadamente em 3 consultas de sua escolha e retorne os top-5 documentos mais similares à cada consulta;

In [0]:
queries = ['juíza federal', 'jair bolsonaro', 'paulo guedes']

In [0]:
db = {query:[] for query in queries}
dtf = {query:[] for query in queries}
dtfidf = {query:[] for query in queries}
dbm25 = {query:[] for query in queries}

for query in queries:
  n = 0
  for doc in result.text:
    doc = doc.lower()
    n += 1
    bisect.insort(db[query], (binary_vsm(query, doc), n))
    bisect.insort(dtf[query], (tf_vsm(query,doc), n))
    bisect.insort(dtfidf[query], (tfidf_vsm(query,doc), n))
    bisect.insort(dbm25[query], (bm25_vsm(query,doc,n), n))
  
  db[query].reverse()
  dtf[query].reverse()
  dtfidf[query].reverse()
  dbm25[query].reverse()

## 5. Compare os resultados encontrados e responda.

### 5.1. Quais modelos você acha que trouxe os melhores resultados? Por que? Inspecione os documentos retornados para melhor embasar sua resposta.

### 5.2. Calcule e reporte o overlap par-a-par entre os resultados de cada modelo (usando o índice de Jaccard)